In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from helper_function import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### 1. Data preparation

In [2]:
train_dir_path = "image_data/101_food_classes_10_percent/train/"
test_dir_path = "image_data/101_food_classes_10_percent/test/"

In [9]:
### Walk thro image data path

walk_through_dir("image_data/101_food_classes_10_percent")

There are 2 directories and 0 images in 'image_data/101_food_classes_10_percent'.
There are 101 directories and 0 images in 'image_data/101_food_classes_10_percent\test'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\apple_pie'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\baby_back_ribs'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\baklava'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\beef_carpaccio'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\beef_tartare'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\beet_salad'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\beignets'.
There are 0 directories and 250 images in 'image_data/101_food_classes_10_percent\test\bibimbap'.
There are 0 directories and

In [6]:
img_size = (224, 224)

### Shuffle for training data to prevent overfitting
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir_path,
    image_size = img_size, label_mode = "categorical", batch_size = 32)

### Don't shuffle testing data, we do not need it on testing
test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir_path,
    image_size = img_size, label_mode = "categorical")

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [3]:
### Create check point

### Save only the model weight
### Save the model`s weight which score the best validation accuracy
checkpoint_path = "101_classes_10_percent_transfer_learning_3_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only = True,
    monitor = "val_accuracy", save_best_only = True)

In [4]:
### Create data augmentation

data_augmentation = Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
  layers.RandomZoom(0.2),
  layers.RandomHeight(0.2),
  layers.RandomWidth(0.2),
], name = "data_augmentation")

In [7]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top = False)
base_model.trainable = False

input_layer = layers.Input(shape = (224, 224, 3), name = "input_layer")

x = data_augmentation(input_layer)
x = base_model(x, training = False)
x = layers.GlobalAveragePooling2D(name = "global_average_pooling")(x)

output_layer = layers.Dense(len(train_data_10_percent.class_names), activation = "softmax", name = "output_layer")(x)

model = tf.keras.Model(input_layer, output_layer)

model.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling (Gl  (None, 1280)              0         
 obalAveragePooling2D)                                           
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                             